<a href="https://colab.research.google.com/github/judem-21/Natural-Language-Processing/blob/main/Sentiment%20Analysis/SAModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##SA Model

###Importing Libraries

In [ ]:
!pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 23.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12

In [ ]:
import pandas as pd
from collections import Counter
import torch
import torch.nn as nn
import torch.optim as optim
from datasets import load_dataset
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from torch.optim.lr_scheduler import StepLR

###Loading and Preprocessing the Dataset

In [ ]:
dataset = load_dataset("Sp1786/multiclass-sentiment-analysis-dataset")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

train_df.csv:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

val_df.csv:   0%|          | 0.00/601k [00:00<?, ?B/s]

test_df.csv:   0%|          | 0.00/586k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/31232 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5205 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5206 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
dataset['train'][9]

{'id': 793, 'text': '  sad face.', 'label': 0, 'sentiment': 'negative'}

In [ ]:
def filter_none(example):
    return example["text"] is not None

dataset['test'] = dataset['test'].filter(filter_none)
dataset['validation']=dataset['validation'].filter(filter_none)
dataset['train']=dataset['train'].filter(filter_none)

Filter:   0%|          | 0/5206 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5205 [00:00<?, ? examples/s]

Filter:   0%|          | 0/31232 [00:00<?, ? examples/s]

In [ ]:
def tokenize(batch):
    tokenised = tokenizer(text=batch["text"], truncation=True)
    tokenised["labels"] = batch["label"]
    return tokenised

tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(['id', 'text', 'sentiment'])

Map:   0%|          | 0/31232 [00:00<?, ? examples/s]

Map:   0%|          | 0/5205 [00:00<?, ? examples/s]

Map:   0%|          | 0/5205 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")
train_loader = DataLoader(
    tokenized_dataset["train"],
    batch_size=32,
    shuffle=True,
    collate_fn=data_collator
)
val_loader=DataLoader(
    tokenized_dataset["validation"],
    batch_size=32,
    shuffle=False,
    collate_fn=data_collator
)

In [ ]:
for batch in val_loader:
    print({k: v.shape for k, v in batch.items()})  # Shows input_ids, attention_mask shapes
    print(f'Input Ids:{batch["input_ids"]}')
    print(f'Attention Mask:{batch["attention_mask"]}')
    print(f'Labels:{batch["labels"]}')
    break

{'input_ids': torch.Size([32, 131]), 'token_type_ids': torch.Size([32, 131]), 'attention_mask': torch.Size([32, 131]), 'labels': torch.Size([32])}
Input Ids:tensor([[  101, 10201,  1999,  ...,     0,     0,     0],
        [  101,  1051,  9541,  ...,     0,     0,     0],
        [  101,  4283,   999,  ...,     0,     0,     0],
        ...,
        [  101,  3666,  5095,  ...,     0,     0,     0],
        [  101,  1035, 10658,  ...,     0,     0,     0],
        [  101,  1996,  6287,  ...,     0,     0,     0]])
Attention Mask:tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
Labels:tensor([0, 2, 2, 2, 2, 2, 1, 1, 1, 0, 0, 1, 2, 0, 1, 2, 0, 0, 0, 1, 0, 2, 1, 0,
        1, 1, 2, 0, 0, 1, 2, 1])


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###Defining the Model

In [ ]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
class SentimentAnalyser(nn.Module):
  def __init__(self,embedding_dim,vocab_size,num_rec_layers,dropout,pad_idx,bidirectional=True):
    super().__init__()
    #self.embedding_dim=embedding_dim
    self.embeddings=nn.Embedding(vocab_size,embedding_dim,padding_idx=pad_idx)
    self.linear=nn.Linear((2 if bidirectional else 1)*embedding_dim,3)
    self.lstm=nn.LSTM(input_size=embedding_dim,num_layers=num_rec_layers,hidden_size=embedding_dim,bidirectional=bidirectional,batch_first=True,dropout=dropout)
    self.dropout=nn.Dropout(dropout)
    self.relu=nn.ReLU()
    self.bidirectional=bidirectional

  def forward(self, input, lengths):
    # input: [batch_size, seq_len]

    embedded = self.embeddings(input)  # [batch_size, seq_len, embedding_dim]

    # Sort by lengths (required by pack_padded_sequence)
    lengths_sorted, perm_idx = lengths.sort(0, descending=True)
    #print(f'perm_idx:{perm_idx}')
    embedded = embedded[perm_idx]
    packed_input = pack_padded_sequence(embedded, lengths_sorted.cpu(), batch_first=True)

    packed_output, (h_n, c_n) = self.lstm(packed_input)

    # Reverse the sort
    _, unperm_idx = perm_idx.sort(0)

    # h_n: [num_layers * num_directions, batch_size, hidden_size]
    # Take the final layer's hidden states
    h_n = h_n.view(self.lstm.num_layers, 2 if self.lstm.bidirectional else 1, embedded.size(0), self.lstm.hidden_size)
    h_last = h_n[-1]  # last layer
    #print(f'h_last.shape:{h_last.shape}')
    if self.lstm.bidirectional:
        h_last = torch.cat((h_last[0], h_last[1]), dim=1)  # [batch, 2*hidden]
    else:
        h_last = h_last.squeeze(0)  # [batch, hidden]

    # Unsort back to original order
    h_last = h_last[unperm_idx]

    out = self.linear(self.dropout(h_last))
    return self.relu(out)


###Training

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
checkpoint_path='/content/drive/MyDrive/SentimentAnalysis/checkpoint.pth'
legend={0:'Negative',1:'Neutral',2:'Positive'}
pad_idx = tokenizer.pad_token_id
model=SentimentAnalyser(embedding_dim=256,vocab_size=tokenizer.vocab_size,num_rec_layers=2,
dropout=0.5,pad_idx=pad_idx, bidirectional=True).to(device)
#checkpoint=torch.load(checkpoint_path)
#model.load_state_dict(checkpoint['model_state_dict'])
softmax=nn.Softmax(dim=1)

In [ ]:
num_epochs,lr=100,3e-4
# Learning rate scheduler: reduce LR by gamma every 'step_size' epochs
optimizer=torch.optim.Adam(model.parameters(),lr=lr)
scheduler = StepLR(optimizer, step_size=10, gamma=0.5)  # halve LR every 5 epochs
loss_fn=nn.CrossEntropyLoss()
#min_loss=checkpoint['min_loss']
min_loss=np.inf
#optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [ ]:
def accuracy(outputs,labels,training=False):
  if not training: return (outputs==labels).sum()/len(labels)
  return (softmax(outputs).argmax(dim=1)==labels).sum()/len(labels)

In [ ]:
epoch_losses=[]
for epoch in range(1,num_epochs+1):
  batch_losses=[]
  model.train()
  for idx,batch in enumerate(train_loader):
    optimizer.zero_grad()
    inputs=batch['input_ids'].to(device)
    labels=batch['labels'].to(device)
    logits=model(input=inputs,lengths=batch['attention_mask'].sum(dim=1))
    loss=loss_fn(logits,labels)
    loss.backward()
    optimizer.step()
    batch_losses.append(loss.item())
    if (epoch==1 or epoch%10==0) and (idx+1)%100==0:
      print(f'Epoch {epoch}, Batch {idx+1}, Batch Loss:{batch_losses[-1]:.8f} and accuracy:{accuracy(logits,labels,training=True)*100}%')
  epoch_losses.append(np.mean(batch_losses))
  scheduler.step()

  model.eval()
  if epoch%5==0:
   with torch.no_grad():
    for idx,batch in enumerate(val_loader):
      inputs=batch['input_ids'].to(device)
      labels=batch['labels'].to(device)
      logits=model(inputs,batch['attention_mask'].sum(dim=1))
      loss=loss_fn(logits,labels)
      print(f'Epoch {epoch}, Batch {idx+1}, Validation Batch Loss:{loss.item():.4f}, and accuracy:{accuracy(logits,labels,training=True)*100}%')

  if epoch==1 or epoch%10==0:
    print(f'Epoch {epoch}, Learning Rate={scheduler.get_last_lr()[0]}')
  print(f'Epoch {epoch}, Epoch Loss:{epoch_losses[-1]:.4f}')
  if epoch_losses[-1]<min_loss:
    min_loss=epoch_losses[-1]
    checkpoint={'model_state_dict':model.state_dict(),'optimizer_state_dict':optimizer.state_dict(),'min_loss':epoch_losses[-1].item()}
    torch.save(checkpoint,checkpoint_path)
    print('Model saved!')
  print()

###Testing

In [ ]:
test_loader=DataLoader(
    tokenized_dataset["test"],
    batch_size=32,
    shuffle=False,
    collate_fn=data_collator
)

In [ ]:
model.eval()
for idx,batch in enumerate(test_loader):
  if (idx+1)%10!=0:continue
  with torch.no_grad():
    logits=model(batch['input_ids'].to(device),batch['attention_mask'].sum(dim=1))
    logits=softmax(logits).argmax(dim=1)
    #print(f'Logits shape: {logits.shape}')
    print(f"Batch: {idx+1}, Accuracy: {accuracy(logits,batch['labels'].to(device))*100}%")

In [ ]:
def determine_sentiment(output,legend):
  return legend[output.argmax(dim=1).item()]

In [ ]:
custom_inputs=["I am unhappy with your behaviour!","This is a good outcome!","I am perfectly fit and fine."]
for custom_input in custom_inputs:
 print(f'{custom_input}: ',end='')
 tokenized=tokenizer(custom_input,return_tensors='pt')
 print(determine_sentiment(model(tokenized['input_ids'].to(device),tokenized['attention_mask'].sum(dim=1)),legend))

I am unhappy with your behaviour!: Negative
This is a good outcome!: Positive
I am perfectly fit and fine.: Neutral
